<a href="https://colab.research.google.com/github/dbremont/Notas/blob/main/Ingenerare/Databases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- SQL Lite Source Code
  - Build & Debug
- Virtual Machines
- Parser
- B-Tree ...
- Para instalar SQLite hay que primero instalar tlc
- Para ma;na tratar de ./configure && make en folder correctamente y ejecutar la shell y crear una tabla simple.

----

- Hacer fork a Simple DB MIT y verificar el codigo.
- Cuantas clases hay,
- Cuantas lineas de codigo hay.
- Cuantos comentario hay.
- Cuantas interfaces hay.
- Cuantas enum hay.

- Puedo compilarlo y generar el jar.
- Puedo usar el jar?

- Resumen de Terminologia.

## Primario


* [15-445/645 Introduction to Database Systems](https://15445.courses.cs.cmu.edu/fall2019/)

* [15-721 Advanced Database Systems](https://15721.courses.cs.cmu.edu/spring2020/)

## Secundario

* [CS6530: Graduate-level Database System](https://www.cs.utah.edu/~lifeifei/cs6530/)

* [Database Systems](https://ocw.mit.edu/courses/electrical-engineering-and-computer-science/6-830-database-systems-fall-2010/)

* [CS 276 / LING 286: Information Retrieval and Web Search](http://web.stanford.edu/class/cs276/)

* [6.830/6.814: Database Systems](http://dsg.csail.mit.edu/6.830/index.php) 

* [6.033 Computer System Engineering](http://web.mit.edu/6.033/www/index.shtml)

* [A Celebration of Mike Stonebraker](https://www.youtube.com/playlist?list=PLSE8ODhjZXjZO802SjzqBFFlkuKglgbZD)

* [How To Compile SQLite](https://sqlite.org/howtocompile.html)

- Relational Databases
- Storage
- Execution
- Concurrency Control
- Recovery
- Distributed Databases
- Potpourri

BusTub

- Disk-based Storage,
- Volcano-style Query Processing,
- Pluggable APIs
- Currently does not support SQL

## Query Plans

Join Algorithms:

- [Join (SQL)](https://en.wikipedia.org/wiki/Join_(SQL))
- [Hash join](https://en.wikipedia.org/wiki/Hash_join)
- [Query plan](https://en.wikipedia.org/wiki/Query_plan)


- Los planes en oracle se leen de abajo /  ariva.
- Cuándo un plan es mejor que otro?
- 

### Understanding the execution plan


- **Cardinality**– Estimate of the number of rows coming out of each of the operations.

- **Access method** – The way in which the data is being accessed, via either a table scan or index access.

- **Join method** – The method (e.g., hash, sort-merge, etc.) used to join tables with each other.

- **Join type** – The type of join (e.g., outer, anti, semi, etc.).

- **Join order** – The order in which the tables are joined to each other.

- **Partition pruning** – Are only the necessary partitions being accessed to answer the query?

- **Parallel Execution** – In case of parallel execution, is each operation in the plan being conducted in parallel? Is the right data redistribution method being used?

### Access Method

The access method - or access path - shows how the data will be accessed from each table (or index). The access method  is shown in the operation field of the explain plan.

Oracle supports nine common access methods:

- **Full table scan :** Reads all rows from a table and filters out those that do not meet the where clause predictates. A full table scan will us muti block IO (typically 1MB IOs). A full table scan is selected if a large portion of the rows in the table must be acessed, no indexes exist or the ones present can't be used or if the cost is the lowest. The decision to use a full table scan is also influenced by the following:

  - Init.ora parameter db_multi_block_read_count
  - Parallel degree
  - Hints
  - Lack of useable indexes
  - Using an index cost more

- **Table access by ROWID:** The rowid of a row specifies the data file, the data block within that file, and the location of the row within that block. Oracle first obtains the rowids either from a *WHERE* clause predicate or through an index scan of one or more of the table's indexes. Oracle then locates each selected row in the table based on its *rowid* an does row-by-row access.

- **Index unique scan** - Only one row will e rturned from he scan of a unique index. It will be used when there is an equiality predictate on a unique (B-tree) index or an index create as a result of a primary key constraint.

- **Index range scan** – Oracle accesses adjacent index entries and then uses the ROWID values in the index to retrieve the corresponding rows from the table. An index range scan can be bounded or unbounded. It will be used when a statement has an equality predicate on a non-unique index key, or a non-equality or range predicate on a unique index key. (=, <, >,LIKE if not on leading edge). Data is returned in the ascending order of index columns.

- **Index range scan descending** – Conceptually the same access as an index range scan, but it is used when an ```ORDER BY .. DESCENDING``` clause can be satisfied by an index.

- **Index skip scan** - Normally, in order for an index to be used, the prefix of the index key (leading edge of the index) would be referenced in the query. However, if all the other columns in the index are referenced in the statement except the first column, Oracle can do an index skip scan, to skip the first column of the index and use the rest of it. This can be advantageous if there are few distinct values in the leading column of a concatenated index and many distinct values in the non-leading key of the index.

- Full Index scan - A full index scan does not read every block in the index structure, contrary to what its name suggests. An index full scan processes all of the leaf blocks of an index, but only enough of the branch blocks to find the first leaf block. It is used when all of the columns necessary to satisfy the statement are in the index and it is
cheaper than scanning the table. It uses single block IOs. It may be used in any of the following situations:

  -  An *ORDER BY* clause has all of the index columns in it and the order is the same as in the index (can also contain a subset of the columns in the index).
  
  - The query requires a sort merge join and all of the columns referenced in the query are in the index.
  
  -  Order of the columns referenced in the query matches the order of the leading index columns.

  - A *GROUP BY* clause is present in the query, and the columns in the *GROUP BY* clause are present in the index. 

- **Fast full index scan** - This is an alternative to a full table scan when the index contains all the columns that are needed for the query, and at least one column in the index key has the NOT NULL constraint. It cannot be used to eliminate a sort operation, because the data access does not follow the index key. It will also read all of the blocks in
the index using multiblock reads, unlike a full index scan.

- **Index join** – This is a join of several indexes on the same table that collectively contain all of the columns that are referenced in the query from that table. If an index join is used, then no table access is needed, because all the relevant column values can be retrieved from the joined indexes. An index join cannot be used to eliminate a sort
operation

- **Bitmap Index** – A bitmap index uses a set of bits for each key values and a mapping function that converts each bit position to a rowid. Oracle can efficiently merge bitmap indexes that correspond to several predicates in a ```WHERE``` clause, using Boolean operations to resolve ```AND``` and ```OR``` conditions.

### Join Method

The join method describes how data from two data producing operators will be joined together. You can identify the join methods used in a SQL statement by looking in the operations column in the explain plan. 

**Hash Joins** - Hash joins are used for joining large data sets. The optimizer uses the smaller of the two tables or data sources to build a hash table, based on the join key, in memory. It then scans the larger table, and performs the same hashing algorithm on the join column(s). It then probes the previously built hash table for each value and if they match, it returns a row.

**Nested Loops joins** - Nested loops joins are useful when small subsets of data are being joined and if there is an efficient way of accessing the second table (for example an index look up). For every row in the first table (the outer table), Oracle accesses all the rows in the second table (the inner table). Consider it like two embedded FOR loops. In Oracle Database 11g the internal implementation for nested loop joins changed to reduce overall latency for physical I/O so it is possible you will see two ```NESTED LOOPS``` joins in the operations column of the plan, where you previously only saw one on earlier versions of Oracle

**Sort Merge joins** – Sort merge joins are useful when the join condition between two tables is an in-equality condition such as, <, <=, >, or >=. Sort merge joins can perform better than nested loop joins for large data sets. The join consists of two steps:

- Sort join operation: Both the inputs are sorted on the join key.

- Merge join operation: The sorted lists are merged together.

**Cartesian join** - The optimizer joins every row from one data source with every row from the other data source, creating a Cartesian product of the two sets. Typically this is only chosen if the tables involved are small or if one or more of the tables does not have a join conditions to any other table in the statement. Cartesian joins are not
common, so it can be a sign of problem with the cardinality estimates, if it is selected for any other reason. Strictly speaking, a Cartesian product is not a join.

### Join Types

Oracle offers several join types: inner join, (left) outer join, full outerjoin, anti join, semi join, grouped outer join, etc. Note that inner join is the most common type of join; hence the execution plan does not specify the key word “INNER’

**Outer Join** - An outer join returns all rows that satisfy the join condition and also all of the rows from the table without the (+) for which no rows from the other table satisfy the join condition. For example, $T1.x = T2.x (+)$, here T1 is the left table whose non-joining rows will be retained. In the ANSI outer join syntax, it is the leading table whose non-join rows will be retained. The same example can be written in ANSI SQL as $T1$ ```LEFT OUTER JOIN``` $T2$ ON $(T1.x = T2.x)$;

### Join Order

The join order is the order in which the tables are joined together in a multi-table SQL statement. To determine the join order in an execution plan look at the indentation of the tables in the operation column. In Figure 22 below the ```SALES``` and ```PRODUCTS``` table are equally indented and both of them are more indented than the ```CUSTOMERS``` table. Therefore the ```SALES``` and ```PRODUCTS``` table will be joined first using a hash join and the result of that join will then be
joined to the ```CUSTOMERS``` table.

The join order is determined based on cost, which is strongly influenced by the cardinality estimates and the access paths available. The Optimizer will also always adhere to some basic rules:

- Joins that result in at most one row always go first. The Optimizer can determine this based on UNIQUE and PRIMARY KEY constraints on the tables.

- When outer joins are used the row preserving table (table without the outer join operator) must come after the other table in the predicate (table with the outer join operator) to ensure all of the additional rows that don’t satisfy the join condition can be added to the result set correctly.

- When a subquery has been converted into an antijoin or semijoin, the tables from the subquery must come after those tables in the outer query block to which they were connected or correlated. However, hash antijoins and semijoins are able to override this ordering condition under certain circumstances.

- If view merging is not possible all tables in the view will be joined before joining to the tables outside the view.



### Partitioning



Partitioning allows a table, index or  index-organized table to be subdivided into smaller pieces. Each piece of the database object is called a Partition. Partition pruning or Partition elimination is the simplest means to improve
performance using Partitioning. For example, if an application has an ```ORDERS``` table that contains a record of all orders for the last 2 years, and this table has been partitioned by day, a query requesting orders for a single week would only access seven partitions of the ```ORDERS``` table instead of 730 partitions (the entire table).

Partition pruning is visible in an execution plan in the ```PSTART``` and ```PSTOP``` columns. The ```PSTART``` column contains the number of the first partition that will be accessed and PSTOP column contains the number of the last partition that will be accessed1

## 01 Relational Model

**Database**: Organized collection of inter-related data that models some aspect of the real-world.

- Ej. Create a database that models a diginal music store to keep track of artists and albumns.

**Database Management System**: Is a software system that allows applications to store and analyze information in a database.  

A general-purpose DBMS is designed to allow the definition, creation, querying, update, and administration of databases.

Database abstraction to avoid this
maintenance:
-  Store database in simple data structures.
- Access data through high-level language,
DBMS figures out best strategy.
-  Physical storage left up to the DBMS
implementation.

Data Models

- A data model is a collection of concepts for
describing the data in a database.
  - Relational,
  - Key/Value,
  - Graph,
  - Document,
  - Column-family,
  - Array/Matrix,
  - Hierarchical,
  - Network,
  - Multi-Value

- A schema is a description of a particular collection
of data, using a given data model.

Relational Data Model

- **Structure**: The Definition of the database's relations and their contents.
- **Integrity**: Ensure the database's contents sastify constrains.
- **Manipulation**: Programming interface for accesing and modifying a database's contents.

### Relational Data Model Concepts

- A Relation: is an unordered set that contain the relationship of attributes that represents entities.
- A tuple: is a set of attribute values (also kknown as its domain) in the relation.
 - Values are (normally) atomic/scalar.
 - The special value NULL is a member of every domain.

Primary Keys: A relation's primary keys uniquely identifies a single tuple.
- Some DBMSs automatically create an internal primary key if a table does not defined one.
- Auto-generation of unique integer primary keys:
  - Sequence (sql:2003)
  - Auto_Increment (MySQL)

Foreign Keys:
- A *foreign key* specifies that an attribute from one relation has to map to a tuple in another relation.

Data Manipulation Languages (DML)

- Methods to store and retrieve information from a database.
- Procedural: The query specifites the (high-level) strategy the DBMS should use to find the desired result. (Relational Algebra)
- Non-Procedural: The query specifies only what data is wanted and not how to find it. (Relational Calculus)

Relational Algebra:

- **Note**: Relational algebra still defines the high-level steps of how to compute a query. A better approach is to state the high-level answer that you want  the DBMS to compute.

- Defines the primitives for processing queries on a relational database.

- Fundamental operations that retrieve and manipulate tuples in a relation.
- Each *operator* takes one or more relation as its inputs and ouputs a new relation. "We can 'chain' operators together to create  more complex operations".


- Select: Choose a subset of tuples from a relation that sastisfies a selection predicate.
- Projection: Generate a relation with tuples that contains only the specifies attributes.
- Union: Generate a relation that constains all tuples that appear in either only one or both input relations.
- Intersection: Generate a relation that contains only the tuples that appear in both of the input relations.
- Difference: Generate a relation that contains only the tuples 
- Product: Generate a relation that contains all possible combinations of tuples from the input relations.
- Join: Generate a relation that contains all tuples that are combination of two tuples(one from each input relation)  which a commmon value(s) from one or more attributes.


Extra Operators:
 - Rename,
 - Assign,
 - Duplicate Elimination,
 - Aggregation,
 - Sorting,
 - Division

Queries:
- The relational model is independent of any query language   implementation. 
- *SQL* is the *de fato* standard (many dialects).

## 02 Intermediate SQL

Relational Languages

- User only needs to specify the answer that they want, not how to compute it.
- Specificy == Describe
- The DBMS is responsible for efficient evaluation of the query.
- High-end systems have a sophisticated "query optimizer" that can rewrite queries and search for optimal execution strategies.

SQL History

- IBM's first query language was called "SQUARE".
- Current standard is SQL:2016
- SQL:2016 - JSON, Polymorphic tables
- SQL:2011 - Temporal DBs, Pipelined DML
- SQL:2008 - Truncation, Fancy Sorting
- SQL:2003 - XML, Windows, Sequences, Auto-Gen IDs.
- SQL:1999 - Regex, Triggers, OO

- Data Manipulation Language (DML)
- Data Definition Language (DDL)
- Data Control Language (DCL)
 
- SQL is based on bags (duplicates) not sets (no duplicates).


Aggregations + Group By
String / Date / Time Operations
Output Control + Redirection
Nested Queries
Common Table Expressions
Window Functions

Aggregates
Functions that return a single value from a bag of tuples:
- AVG(col)→ Return the average col value.
- MIN(col)→ Return minimum col value.
- MAX(col)→ Return maximum col value.
- SUM(col)→ Return sum of values in col.
- COUNT(col)→ Return # of values for col.
- Distinct Aggregates : COUNT, SUM, AVG support DISTINCT


Group By
- Project tuples into subsets and calculate aggregates against each subset. 

Having
- Filters results based on aggregation computation.

String Operations
SQL-92   | Sensitive 	| Single  Only
Postgres | Sensitive	| Single Only
MySQL    | Insensitive | Single/Double
SQLite   | Sensitive   | Single/Double
DB2      | Sensitive   | Single Only
Oracle   | Sensitive   | Single Only

- LIKE is used for string matching. 
  - String-matching operators
	  - '%' Matches any substring (including empty strings).
	  - '_' Match any one character
 - SQL standard says to use || operator to concatenate two or more strings together.

- DATE/TIME OPERATIONS  

- Operations to manipulate and modify DATE/TIME attributes

Output Redirection

Store query results in another table
- Table must not already be defined.
- Table wil have the same # columns with the same types as the input.
- INSERT INTO CourseIds (SELECT DISTINCT cid FROM enrolled); SQL92

Output Control
- Order BY <column*> [ASC | DESC]
- SELECT sid, grade FROM enrolled WHERE cid = '15-721' ORDER BY grade
- LIMIT <count> [offset]

Nested Queries
- Queries containing other queries.
- They are often difficult to optimize.
- Inner queries can appear (almost) anywhere in query.
- SELECT name FROM student WHERE sid IN (SELECT sid FROM enrolled)
- ALL - Must satisfy expression for all rows in the sub-query.
- ANY - Must satisfy expression for at least one row in the sub-query.
- IN  - Equivalent to '=ANY()' .
- EXISTS -  At least one row is returned.

Window Functions
- Performs a "sliding" calculation across a set of tuples that are related.
- Like an aggregation but tuples are not grouped into a single output tuples.
- SELECT ... FUNC-NAME(...) OVER (...) FROM tableName
- FUNC-NAME: Aggregation Functions Special Functions
- OVER: How to "slice" up data - Can also sort
   - Use PARTITION BY to specify group.  OVER (PARTITION BY cid)
- Aggregation functions: Anything that we discussed earlier
- Special window functions: 
   - ROW_NUMBER() :  # of the current row
   - RANK() :  Order position of the current row.
- SELECT cid, sid, ROW_NUMBER() OVER (PARTITION BY cid) FROM enrolled ORDER BY cid 

Common Table Expressions

- Provides a way to write auxiliary statements for use in a larger query.
- WITH cteName AS (
	SELECT 1
)
SELECT * FROM cteName

CTE Recursion
- Print the sequence of numbers from 1 to 10
- WITH RECURSIVE cteSource (counter) AS (
(SELECT 1)
UNION ALL
(SELECT counter + 1 FROM cteSource
WHERE counter < 10)
)
SELECT * FROM cteSource

We now understand what a database looks like at a logical level and how to write queries to read/write data (e.g., using SQL).

We will next learn how to build software that manages a database (i.e., a DBMS).

## 03-04 Database Storage 

**Disk-Based Architecture**

- The DBMS assumes that the primary storage location of the database is on non-volatile disk.

- The DBMS's components manage the movement of data between non-volatile and volatile storage.

![Img](https://upload.wikimedia.org/wikipedia/commons/thumb/0/0c/ComputerMemoryHierarchy.svg/800px-ComputerMemoryHierarchy.svg.png "abc")

[Latency numbers every programmer should know](https://gist.github.com/hellerbarde/2843375)

### Latency numbers every programmer should know
    L1 cache reference ......................... 0.5 ns
    Branch mispredict ............................ 5 ns
    L2 cache reference ........................... 7 ns
    Mutex lock/unlock ........................... 25 ns
    Main memory reference ...................... 100 ns             
    Compress 1K bytes with Zippy ............. 3,000 ns  =   3 µs
    Send 2K bytes over 1 Gbps network ....... 20,000 ns  =  20 µs
    SSD random read ........................ 150,000 ns  = 150 µs
    Read 1 MB sequentially from memory ..... 250,000 ns  = 250 µs
    Round trip within same datacenter ...... 500,000 ns  = 0.5 ms
    Read 1 MB sequentially from SSD* ..... 1,000,000 ns  =   1 ms
    Disk seek ........................... 10,000,000 ns  =  10 ms
    Read 1 MB sequentially from disk .... 20,000,000 ns  =  20 ms
    Send packet CA->Netherlands->CA .... 150,000,000 ns  = 150 ms


Assuming ~1GB/sec SSD

![Visual representation of latencies](http://i.imgur.com/k0t1e.png)

Visual chart provided by [ayshen](https://gist.github.com/ayshen)

Data by [Jeff Dean](http://research.google.com/people/jeff/)

Originally by [Peter Norvig](http://norvig.com/21-days.html#answers)

**Sequential VS. Random Access**

Random access on non-volatile storage is usually much slower than sequential access.

DBMS will want to maximize sequential access.
-  Algorithms try to reduce number of writes to random pages so that data is stored in contiguous blocks.
-  Allocating multiple pages at the same time is called an extent.

**System Design Goals**

- Allow the DBMS to manage databases that exceed the amount of memory available.
- Reading/writing to disk is expensive, so it must be managed carefully to   void large stalls and performance degradation.
- Random access on disk is usually much slower than sequential access, so the DBMS will want to maximize sequential access.

**Disk-Oriented DBMS**


- Database File

![Img](https://raw.githubusercontent.com/dbremont/Notas/main/assets/img-database-file-01.jpg)

- Buffer Pools
- Execution Engine
  - Get page #2 to teh buffer pool, if the page is not in the pool it must(veriricar) be put in the pool. Return back a pointer to page #2

**Why Not Use The OS?**

- The DBMS can use memory mapping (mmap) to store the contents of a file into the address space of a program.

- The OS is responsible for moving the pages of the file in and out of momory, os the DBMS does't need to worry about it.

- What if we allow multiple threads to access the mmap files to hide page fault stalls?

- This works good enough for read-only access.
- It is complicated when there are multiple writers…

- Virtual Memory , Physical Memory

- **TODO** Replicate **nmap** claim.

There are some solutions to this problem:
- madvise: Tell the OS how you expect to read certain pages.
- mlock: Tell the OS that memory ranges cannot be paged out.
- msync: Tell the OS to flush memory ranges out to disk.

Full nmap usage:
- monetdb,
- lmdb,
- revenddb,
- leveldb,
- elasticsearch,

Partial Use:
- mongoDB,
- memsql,
- sqlite,
- influxdb

DBMS (almost) always wants to control things
itself and can do a better job than the OS.

-  Flushing dirty pages to disk in the correct order.
- Specialized prefetching.
- Buffer replacement policy.
- Thread/process scheduling.
- The OS is **not** your friend.

**Database Storage**

- Problem 1: How the DBMS represents the database in files on disk.
- Problem 2: HOw the DBMS manages its memory and moves data back-and-forth from disk.

- File Storage
- Page Layout
- Tuple Layout


The DBMS stores a database as one or more files on disk typically in a proprietary format.
-  The OS doesn't know anything about the contents of these files.

Early systems in the 1980s used custom filesystems on raw storage.
- Some "enterprise" DBMSs still support this.
- Most newer DBMSs do not do this.

**Storage Manager**

The storage manager is responsible for maintaining a database's files.
- Some do their own scheduling for reads and writes to improve spatial and temporal locality of pages.

It organizes the files as a collection of pages.
- Tracks data read/written to pages.
- Tracks the available space.

**Database Pages**

A page is a fixed-size block of data.
- It can contain tuples, meta-data, indexes, log records…
- Most systems do not mix page types.
- Some systems require a page to be self-contained.

Each page is given a unique identifier.
- The DBMS uses an indirection layer to map page IDs to physical locations.

There are three different notions of
"pages" in a DBMS:
- Hardware Page (usually 4KB)
- OS Page (usually 4KB)
- Database Page (512B-16KB)

A hardware page is the largest block of data that the storage device can guarantee failsafe writes.

4kb
- sqlite,
- ibm db2,
- oracle

8kb
- microsoft sql server,
- porgresql

16kb
- mysql

**Database Heap**

A *heap file* is an unordered collection of pages with tuples that are stored in random order.
- Create / Get / Write / Delete Page
- Must also support iterating over all pages.

Two ways to represent a heap file:
* Linked List
* Page Directory

- It is easy to find pages if there is only a single heap file.
- Need meta-data to keep track of what pages exist in multiple files and which ones have free space

Link List

- Maintain a header page at the beginning of the file that stores two pointers:
  - HEAD of the free page list.
  - HEAD of the data page list.

- Each page keeps track of how many free slots they currently have.

Page Directory

- The DBMS maintains special pages that tracks the location of data pages in the database files.

- The directory also records the number of free slots per page.
- Must make sure that the directory pages are in sync with the data pages.

**Page Header** 

Every page contains a header of metadata about the page's contents.
- Page Size
- Checksum
- DBMS Version
- Transaction Visibility
- Compression Information

- Some systems require pages to be selfcontained (e.g., Oracle).

- ![Page Header](https://raw.githubusercontent.com/dbremont/Notas/main/assets/img-page-02.jpg)


**Page Layout**

For any page storage architecture, we now need to decide how to organize the data inside of the page.

- We are still assuming that we are only storing tuples.

Two approaches:
- Tuple-oriented
- Log-structure

How to store tuples in a page?
**Strawman Idea**: Keep track of the number of tuples in a page and then just append a new tuple to the end.

- What happens if we delete a tuple?
- What happens if we have a variablelength attribute?

**Slotted Pages**

- The most common layout scheme is called *slotted pages*.
- The slot array maps "slots" to the tuples' starting position offsets.
- The header keeps track of:

  - The # of used slots
  - The offset of the starting location of the last slot used.

**Record IDS**

- The DBMS needs a way to keep track of individual tuples.

- Each tuple is assigned a unique record identifier.
  -  Most common: page_id + offset/slot
  - Can also contain file location info.

- An application cannot rely on these IDs to mean anything.

**Tuple Layout**

- A tuple is essentially a sequence of bytes.
- It's the job of the DBMS to interpret those bytes into attribute types and 
values.
- Each tuple is prefixed with a header that contains meta-data about it. 
  - Visibility info (concurrency control)
  - Bit Map for NULL values.
- We do **not** need to store meta-data about the schema.
- Attributes are typically stored in the order that you specify them when you 
create the table.
- This is done for software engineering reasons (i.e., simplicity).
- However, it might be more efficient to lay them out differently.

![Tuple Structure](https://raw.githubusercontent.com/dbremont/Notas/main/assets/img-tuple-03.jpg)

DBMS can physically denormalize (e.g., "pre join") related tuples and store them together in the same page. 
- Potentially reduces the amount of I/O for common workload patterns.
- Can make updates more expensive.

```sql
CREATE TABLE foo (
a INT PRIMARY KEY,
b INT NOT NULL,
);

CREATE TABLE bar (
c INT PRIMARY KEY,
a INT
⮱REFERENCES foo (a),
);
```

- ```foo Header a b c c c …```
- foo: ```a b```
- bar: ```c c c …```

- Not a new idea.
  -  IBM System R did this in the 1970s.
  -  Several NoSQL DBMSs do this without calling it physical denormalization.
  - Rethinkdb,
  - couchdb,
  - marklogic,
  - ravendb,
  - mongodb

- Database is organized in pages.
- Different ways to track pages.
- Different ways to store pages.
- Different ways to store tuples.

**Log-Structured File Organization**

- Instead of storing tuples in pages, the DBMS only stores log records.
- The system appends log records to the file of how the database was modified:
  - Inserts store the entire tuple.
  - Deletes mark the tuple as deleted.
  - Updates contain the delta of just the attributes that were modified.

![Page with log structure](https://raw.githubusercontent.com/dbremont/Notas/main/assets/img-log-structure-04.jpg)

- To read a record, the DBMS scans the log backwards and "recreates" the tuple to find what it needs.

- Build indexes to allow it to jump to locations in the log.

- Periodically compact the log.

  - apache hbase,
  - cassandra,
  - leveldb,
  - rocksdb
 
- Compaction coalesces larger log files into smaller files by removing unnecessary records.

**Tuple Storage**

A tuple is essentially a sequence of bytes. It's the job of the DBMS to interpret those bytes into attribute types and values.

The DBMS's catalogs contain the schema information about tables that the system uses to figure out the tuple's layout.

**Data Representation**

- ```INTEGER/BIGINT/SMALLINT/TINYINT```(C/C++ Representation)

- ```FLOAT/REAL vs. NUMERIC/DECIMAL```  (IEEE-754 Standard / Fixed-point Decimals)

- ```VARCHAR/VARBINARY/TEXT/BLOB```     (Header with length, followed by data bytes)

- ```TIME/DATE/TIMESTAMP``` (32/64-bit integer of (micro)seconds since Unix epoch)

**Variable Precision Numbers**

- Inexact, variable-precision numeric type that uses the "native" C/C++ types.
  - Examples: FLOAT, REAL/DOUBLE

- Store directly as specified by IEEE-754.

```c
#include <stdio.h>
int main(int argc, char* argv[]) {
  float x = 0.1;
  float y = 0.2;
  printf("x+y = %f\n", x+y);
  printf("0.3 = %f\n", 0.3);
}

x+y = 0.300000
0.3 = 0.300000
```

**Fixed Precision Numbers**

Numeric data types with (potentially) arbitrary precision and scale. Used when  rounding errors are unacceptable.
- NUMERIC, 
- DECIMAL

Many different implementations.
- Example: Store in an exact, variable-length binary
representation with additional meta-data.
- Can be less expensive if you give up arbitrary precision.

**Postgress - Numeric**

```c
typedef unsigned char NumericDigit;
typedef struct {
  int ndigits; // # of Digits
  int weight;  // Weight of 1st Digit
  int scale;  // Scale Factor
  int sign;  // Positive/Negative/NaN
  NumericDigit *digits; // // Digit Storage
} numeric;
```

**MySQL Numeric**

```c
typedef int32 decimal_digit_t;
struct decimal_t {
  int 
    intg,  // # of Digits Before Point
    frac,  // # of Digits After Point 
    len;   // Length (Bytes)
  bool sign; // Positive/Negative
  decimal_digit_t *buf; // Digit Storage
};
```

**Larges Values**

- Most DBMSs don't allow a tuple to exceed the size of a single page.
- To store values that are larger than a page, the DBMS uses separate
overflow storage pages.
  - Postgres: TOAST (>2KB)
  - MySQL: Overflow (>½ size of page)
  - SQL Server: Overflow (>size of page)

**External Value Storage**

Some systems allow you to store a really large value in an external file.

Treated as a BLOB type.
- Oracle: BFILE data type
- Microsoft: FILESTREAM data type

The DBMS cannot manipulate the contents of an external file.
- No durability protections.
- No transaction protections.

**System Catalogs**

A DBMS stores meta-data about databases in its internal catalogs.
- Tables, columns, indexes, views
- Users, permissions
- Internal statistics

Almost every DBMS stores the database's catalog inside itself (i.e., as tables).
- Wrap object abstraction around tuples.
- Specialized code for "bootstrapping" catalog tables.

You can query the DBMS’s internal *INFORMATION_SCHEMA* catalog to get info about
the database. 
- ANSI standard set of read-only views that provide info about all the tables, views, columns, and procedures in a database

DBMSs also have non-standard shortcuts to retrieve this information.

**Accessing Table Schema**

```sql
SELECT *
FROM INFORMATION_SCHEMA.TABLES
WHERE table_catalog = '<db name>';
```

**Database Workloads**

On-Line Transaction Processing (OLTP)
- Fast operations that only read/update a small amount of data each time.

On-Line Analytical Processing (OLAP)
- Complex queries that read a lot of data to compute aggregates.

Hybrid Transaction + Analytical Processing
- OLTP + OLAP together on the same database instance

![workoad graph](https://raw.githubusercontent.com/dbremont/Notas/main/assets/img-workload-05.jpg)

[Article](https://cacm.acm.org/magazines/2011/6/108651-10-rules-for-scalable-performance-in-simple-operation-datastores/fulltext)

**OLTP**

On-line Transaction Processing:
- Simple queries that read/update a small amount of data that is related to a single entity in the database.

- This is usually the kind of application that people build first.

**OLAP**


On-line Analytical Processing:
- Complex queries that read large portions of the database spanning multiple entities.

You execute these workloads on the data you have collected from your OLTP application(s).

**Data Storage Models**

The DBMS can store tuples in different ways that are better for either OLTP or OLAP workloads.

We have been assuming the n-ary storage model (aka "row storage") so far this semester.

**N-ARY Storage Model (NSM)**

- The DBMS stores all attributes for a single tuple contiguously in a page.

- Ideal for OLTP workloads where queries tend to operate only on an individual entity and insertheavy workloads.

- The DBMS stores all attributes for a single tuple contiguously in a page.

```sql
SELECT * FROM useracct
WHERE userName = ?
AND userPass = ?
```

- Index maps of data to locations.

Advantages

- Fast inserts, updates, and deletes.
- Good for queries that need the entire tuple.

**Decomposition Storage Model (DSM)** "column store"

The DBMS stores the values of a single attribute for all tuples contiguously in a page. 

- Ideal for OLAP workloads where read-only queries perform large scans over a subset of the table’s attributes.

**Tuple Indentification**

- Fixed-length Offsets
- Embedded Tuple Ids

Advantages
- Reduces the amount wasted I/O because the DBMS only
reads the data that it needs.
- Better query processing and data compression (more on
this later).

Disadvantages
- Slow for point queries, inserts, updates, and deletes
because of tuple splitting/stitching

The storage manager is not entirely independent
from the rest of the DBMS.

It is important to choose the right storage model for the target workload:
- OLTP = Row Store
- OLAP = Column Store

## 05 